# Exploration des données - feature engineering
Created by: Thomas Durand-Texte, Feb. 2023

# Import des packages et données
## import des packages

In [ ]:
import os

import pandas as pd
from pandas import IndexSlice as idx


import numpy as np
# import dask as dd
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
import datetime as dt
import scipy.stats as st

import missingno as msno

import pingouin as pg
from sklearn import linear_model
from sklearn import model_selection, metrics, preprocessing


import matplotlib.pyplot as plt
import seaborn as sns

cm = 1./2.54

## Paramètres graphiques et fonctions utiles

In [ ]:
import subprocess

white_font = True
def set_theme( white_font=True ):
    """ set_theme( white_font=True ) """
    if white_font: wht, grey, blck = '0.84' , '0.5', 'k'
    else: wht, grey, blck = 'k', '0.5', '0.84'
    rc = { 'figure.facecolor':(0.118,)*3,
            'axes.labelcolor':wht,
            'axes.edgecolor':wht,
            'axes.facecolor':(0,0,0,0),
            'text.color':'white',
            'text.usetex':False,
            'text.latex.preamble':r'\usepackage[cm]{sfmath} \usepackage{amsmath}' ,
            'font.family': 'sans-serif' ,
            'font.sans-serif': 'DejaVu Sans' ,
            'xtick.color':wht,
            'ytick.color':wht,
            "axes.grid" : True,
            "grid.color": (0.7,)*3,
            "grid.linewidth": 0.4,
            "grid.linestyle": (10,5),
            'legend.edgecolor':'0.2',
            'legend.facecolor':(0.2,0.2,0.2,0.6),
            # 'legend.framealpha':'0.6',
            'pdf.fonttype':42,
            'savefig.format':'pdf',
            'savefig.transparent':True,
            'figure.dpi':150, # for better agreemet figsize vs real size
        }

    base_palette = sns.color_palette()
    sns.set_theme( 'notebook' , rc=rc )
    sns.set_palette( base_palette )
    return


def make_folder( path_folder ):
    path_folder = path_folder.__str__()
    try:
        if os.path.isdir( path_folder ) : return
        os.makedirs(path_folder)
    except OSError:
        pass
    return

def concat_folders(*args, **kwargs):
    """ concat_folders(*args, **kwargs)
        concatenate folders in args (strings) """
    sPath = ''
    for arg in args:
        if arg == '..': sPath = sPath[:sPath[:-1].rfind(os.sep)+1]
        else: sPath += arg
        if sPath[-1] != os.sep: sPath += os.sep
    return sPath

class Path(object):
    """ Path( s_in='', s_lim=None)
        create a path to the string s_in (default is current path)
        and stops after s_lim """
    n_Path = 0
    def __init__(self, s_in='', s_lim=None):
        """docstring."""
        if s_in == '': s_in = os.getcwd()
        if not s_lim is None:
            if s_lim in s_in:
                s_in = s_in[ :s_in.index( s_lim ) + len(s_lim) ]
        self.sPath = concat_folders(s_in)
        self.N = Path.n_Path
        Path.n_Path += 1

    def __add__(self, other):
        """ Path + str : return str """
        if isinstance(other, str): return self.sPath + other

    def __truediv__(self, other):
        """ Path / str : return path concatenated"""
        if isinstance(other, str): return Path(concat_folders(self.sPath, other))

    def __invert__(self):
        """ ~Path : return str of the path """
        return self.sPath

    def __str__(self):
        """ __str__ return str of the path """
        return self.sPath
    # __str__ #

    def makedir( self ):
        return make_folder( self )


def gs_opt( filename ):
    """ otpimisation of a pdf file with gosthscript """
    filenameTmp = filename.replace('.pdf', '') + '_tmp.pdf'
    gs = ['gs',
            '-sDEVICE=pdfwrite',
            '-dEmbedAllFonts=true',
            '-dSubsetFonts=true',             # Create font subsets (default)
            '-dPDFSETTINGS=/prepress',        # Image resolution
            '-dDetectDuplicateImages=true',   # Embeds images used multiple times only once
            '-dCompressFonts=true',           # Compress fonts in the output (default)
            '-dNOPAUSE',                      # No pause after each image
            '-dQUIET',                        # Suppress output
            '-dBATCH',                        # Automatically exit
            '-sOutputFile='+filenameTmp,      # Save to temporary output
            filename]                         # Input file

    subprocess.run(gs)                                      # Create temporary file
    subprocess.run( 'rm -f ' + filename, shell=True)            # Delete input file
    subprocess.run( 'mv -f ' + filenameTmp + " " + filename, shell=True) # Rename temporary to input file

def savefig( fig, savename, **kwargs ):
    """ savefig( fig, savename, **kwargs )
        Saves a figure with kwargs (fig.savefig( savename, **kwargs) ).
        A check is done first to determine if a folder has to be created according to savename.
        Finally, if the file is saved as .pdf, gosthscript optimisation is performed. """
    if os.sep in savename: make_folder( savename[:savename.rindex(os.sep)] )
    fig.savefig( savename, **kwargs )
    savename += '.pdf'
    if os.path.isfile( savename ): gs_opt( savename )


def image_size_from_width_and_shape( width: float, shape: tuple, ymargin=0. ):
    """ return tuple (width, height) corresponding to image shape """
    return width, width*shape[0]/shape[1]+ymargin

def image_size_from_height_and_shape( height: float, shape: tuple, xmargin=0. ):
    """ return tuple (width, height) corresponding to image shape """
    return height*shape[1]/shape[0]+xmargin, height


set_theme()
del set_theme

## Chargement des données

Affichage de l'arborescence

In [ ]:
def print_listdir( path=None, level=0, exclude=['ressources']) :
    suffix = ''
    if level > 0:
        suffix = ' |-'* level
    vals = os.listdir( path )
    vals.sort()
    if path is None:
        path = ''
    for val in vals:
        if val in exclude: continue
        print( suffix, val)
        if os.path.isdir( path + val):
            print_listdir( path + val + '/', level+1 )

print_listdir( exclude=['.venv', 'ressources'] )

1. Chargement des données
2. lower strings
3. compression et sauvegarde des données

In [ ]:
path = 'data/source/'
filename = '2016_Building_Energy_Benchmarking'
compression = 'gzip'

if True:
    df = pd.read_csv( path + filename + '.csv' )
    for key in df.keys():
        if df[key].dtype == 'object':
            df[key] = df[key].str.lower()

    # suppression des colonnes vides (ici seulement comments)
    df.drop( columns=df.keys()[df.isna().sum(0) == len(df)], inplace=True )

    df.to_pickle( r'{:}{:}.pkl'.format(path, filename), compression=compression)
else:
    df = pd.read_pickle( r'{:}{:}.pkl'.format(path, filename), compression=compression )

del compression

In [ ]:
df.columns

# 1. Nettoyage

## 1.1 Initial Filtering : contexte = usage non résidentiel

value counts des 'BuildingType'

In [ ]:
display( df['BuildingType'].value_counts() )

Suppression des Mutlifamily

In [ ]:
sr_loc = df['BuildingType'].str.contains('multifamily')


fig, ax = plt.subplots()
ax.plot( df.loc[~sr_loc,'Longitude'], df.loc[~sr_loc,'Latitude'], 'ro', markersize=4, label='residential')
ax.plot( df.loc[sr_loc,'Longitude'], df.loc[sr_loc,'Latitude'], 'bo', markersize=4, label='others')
ax.legend()
ax.axis('equal')
ax.set_title('Buildind locations')
ax.set_xlabel('Longutide (centered around average)')
ax.set_ylabel('Latitude (centered around average)')


sr_loc = sr_loc[sr_loc]
df.drop( index=sr_loc.index, inplace=True )

In [ ]:
display( df['BuildingType'].value_counts() )

Définition d'une fonction pour déterminer les éléments correspondant à du `housing`

In [ ]:
def is_housing( sr ):
    sr_loc = (sr.str.contains('hous|multifamily', na=False)) \
        & (~sr.str.contains('warehouse|courthouse', na=True))
    return sr_loc

Vérification et suppression à partir du `PrimaryPropertyType`

In [ ]:
var = 'PrimaryPropertyType'
sr_loc = is_housing(df[var])
display( df.loc[sr_loc, var].value_counts() )
df.drop( index=sr_loc[sr_loc].index, inplace=True )

1. Vérification et suppression pour le `LargestPropertyUseType`
2. Vérification des LargestPropertyUseType

In [ ]:
vars = ['LargestPropertyUseType',
        'SecondLargestPropertyUseType',
        'ThirdLargestPropertyUseType']


sr_loc = is_housing( df[vars[0]] )
df.drop( index=sr_loc[sr_loc].index, inplace=True )

sr_loc = is_housing( df[vars[1]] )
for var in vars[2:]:
    sr_loc = sr_loc | is_housing( df[var] )

# display( df['LargestPropertyUseType'].value_counts() )
# display( df['SecondLargestPropertyUseType'].value_counts() )
# display( df['ThirdLargestPropertyUseType'].value_counts() )

display( f'housing find in {sr_loc.sum()} elements' )
display( df.loc[sr_loc, vars] )

del vars, var

## 1.2 Outlier et DefaultData

In [ ]:
def value_counts( sr ):
    value_counts = sr.value_counts()
    return pd.DataFrame( {'count': value_counts.values, 
                        '%':value_counts.values*(100/len(sr)) },
                        index=value_counts.index)

print('Outlier:')
display( value_counts( df['Outlier'] ) )
print('DefaultData:')
display( value_counts( df['DefaultData'] ) )

In [ ]:
df['Outlier'] = df['Outlier'].map( {'low outlier':-1, 'high outlier':1}).fillna(0).astype('int')

## 1.3 Vérifications doublons

In [ ]:
print("Number of duplicated: {:}".format( df['OSEBuildingID'].duplicated().sum() ) )

## 1.4 Variables "inutiles"

Vérification des variables inutiles: création d'une liste "à supprimer"

In [ ]:
vars_to_delete = ['OSEBuildingID', 'PropertyName',
        'TaxParcelIdentificationNumber',
        'CouncilDistrictCode']
# df.drop( columns=vars, inplace=True )

In [ ]:
vars_to_check = ['DataYear', 'City', 'State']

for var in vars_to_check:
    if not var in df.keys() :
        print( f'{var} not in DataFrame')
        continue
    sr = df[var].value_counts()
    display(sr)
    if len(sr) < 2:
        # df.drop( columns=var, inplace=True )
        vars_to_delete.append( var )
        print( f'{var} added to the drop list')

Variable de localité: on prend la variable Neighborhood

In [ ]:
vars_to_delete += ['ZipCode', 'Longitude', 'Latitude']

mask = 'Neighborhood'
display( df[mask].value_counts() )

In [ ]:
df.loc[ df[mask] == 'delridge neighborhoods', mask ] = 'delridge'
display( df[mask].value_counts() )

## 1.5 Recherche et gestion des NaN
### 1.5.1 Détection des NaN
- Pour les Second & Third LargestPropertyUseType : uniquement 1 ou 2 utilisation du bâtiment
- Pour le YearsENERGYSTARCertified : pas de certification
- Pour le ENERGYSTARscore : essayer de le modéliser ?

In [ ]:
sum_isna = df.isna().sum()
print( 'sum isna > 0:' )
display( sum_isna[sum_isna > 0])

In [ ]:
ax = msno.bar( df )

ax = msno.matrix( df.sort_values( by=['SecondLargestPropertyUseType','ThirdLargestPropertyUseType'] ) )


Il y a clairement des lignes avec un gros manque d'informations -> à enlever
- sur les energy : fillna(0) + sum(1) == 0 -> drop
- LargestPropertyUse : à remplir à partir du PrimaryPropertyType 

In [ ]:
energy_keys = ['SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)',
'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)',
'SteamUse(kBtu)', 
'Electricity(kWh)', 'Electricity(kBtu)',
'NaturalGas(therms)', 'NaturalGas(kBtu)',
'TotalGHGEmissions', 'GHGEmissionsIntensity']

property_use_keys = [ 'PrimaryPropertyType', 'ListOfAllPropertyUseTypes', 'LargestPropertyUseType']

others = ['NumberofBuildings']

keys = property_use_keys + others + energy_keys

sr_loc = df[keys].isna().sum(1) > 0
print('Entries with empty cells:')
display( df.loc[sr_loc, keys])

# DROP DATA WITHOUT ENERGY INFORMATION / CONSUMPTION
indexes = (df[energy_keys].fillna(0.).sum(1) == 0.)
indexes = indexes[indexes].index
print('indexes to drop:', indexes.values)
df.drop( index=indexes , inplace=True )

print('Entries with empty cells (after drop):')
sr_loc = df[keys].isna().sum(1) > 0
display( df.loc[sr_loc, keys])

### 1.5.2 Remplissage des NaN
On regarde les éléments sans LargestPropertyUseType

In [ ]:
vars = ['PrimaryPropertyType', 'ListOfAllPropertyUseTypes',
        'LargestPropertyUseType', 'LargestPropertyUseTypeGFA', 'PropertyGFABuilding(s)',
        'PropertyGFAParking', 'PropertyGFATotal' ]
sr_loc = (df['LargestPropertyUseType'].isna())
display( df.loc[sr_loc,vars])

In [ ]:
display( df.loc[ df['PrimaryPropertyType']=="self-storage facility", 'LargestPropertyUseType'].value_counts() )
display( df.loc[ df['PrimaryPropertyType'].str.contains('office'), 'LargestPropertyUseType'].value_counts() )

À priori on peut replacer les valeurs manquantes de "LargestPropertyUseType(GFA)" par la "PrimaryPropertyType" / le "PropertyGFABuilding(s)".

Note: si présence de "office" dans le "PrimaryPropertyType", alors la valeur est "office" (pas de distinctions pour les "LargestPropertyUseType")

In [ ]:
for index in sr_loc[sr_loc].index:
    Primary = df.at[index, 'PrimaryPropertyType']
    df.at[index, 'LargestPropertyUseTypeGFA'] = df.at[index, 'PropertyGFABuilding(s)']
    if 'office' in Primary:
        df.at[index, 'LargestPropertyUseType'] = 'office'
        continue
    df.at[index, 'LargestPropertyUseType'] = Primary

On vérifie le résultat:

In [ ]:
display( df.loc[sr_loc,vars] )

In [ ]:
sum_isna = df.isna().sum()
print( 'sum isna > 0:' )
display( sum_isna[sum_isna > 0])

Remplissage du ZipCode à partir de la longitude et de la latitude: toutes les données étant à Seattle, les variables `Longitude` et `Latitude` peuvent être utilisée comme (`x`,`y`).

Si il y avait plus d'écarts entre les positions, possibilité d'utilisé le package `haversine`.

In [ ]:
sr_loc = df['ZipCode'].isna()
for index in sr_loc[sr_loc].index:
    x,y = df.loc[index, ['Longitude', 'Latitude']].values
    for value in df['Longitude']:
        if isinstance( value, str):
            print('value:', value)
    df['Longitude'].values-x
    df['Latitude'].values-y
    argsort = ((df['Longitude'].values-x)**2 + (df['Latitude'].values-y)**2).argsort()
    for i in argsort[1:]: # neglect the first as it corresponds to the current index
        if np.isnan( df['ZipCode'].iloc[i] ) :
            continue
        df.at[index, 'ZipCode'] = df['ZipCode'].iloc[i]
        break 
    # print('\nx,y:', x,y, '\nx2,y2:', df[['Longitude','Latitude']].iloc[i,:].values )

sum_isna = df.isna().sum()
print( 'After procees sum isna > 0:' )
display( sum_isna[sum_isna > 0])

***
# 2. Vérifications des données d'entrée

In [ ]:
df.keys()

## 2.1 Value_counts

In [ ]:
len(df)

In [ ]:
vars = ['BuildingType', 'PrimaryPropertyType',
       'Neighborhood', 'NumberofBuildings',
       'NumberofFloors', 'LargestPropertyUseType',
        'SecondLargestPropertyUseType',
        'ThirdLargestPropertyUseType',
       ]
for var in vars:
    print( f'{var} value_counts:')
    display( df[var].value_counts() )

## 2.2 Number of floors
On regarde les number of floors pour voir si il y a des valeurs aberrantes

In [ ]:
X, Y = 'PropertyGFABuilding(s)', 'NumberofFloors'

fig, ax = plt.subplots( figsize=(12*cm,12*cm) )
ax.plot( df[X], df[Y], 'bo', markersize=3 )

ax.set_ylabel( 'Number of floors' )
ax.set_xlabel( 'Property GFA buildings (sf)' )

index_100 = df[Y] > 90
index_100 = index_100[index_100].index.values[0]

ax.annotate( 'valeur aberrantes :\n{:}'.format( 
                df.at[index_100, 'PropertyName'] ),
                xy=[0, 99], xytext=[2.1e6, 82],# ha='left', va='center',
                arrowprops=dict(shrink=0.05) )

On assigne la valeur la plus utilisée pour NumberofFloors: 1, sachant que la surface est relativement faible.

In [ ]:
df.at[index_100, Y] = 1

## 2.3 histogramme / transformation

In [ ]:
vars = ['YearBuilt', 'PropertyGFATotal',
       'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'LargestPropertyUseTypeGFA', 
       'SecondLargestPropertyUseTypeGFA', 
       'ThirdLargestPropertyUseTypeGFA']

exponents = [1, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5]

display( df[vars].describe() )
for var, expo in zip(vars, exponents):
    values = df[var].dropna()
    n_zeros = (values==0).sum()
    values = values[values !=0]
    kurtosis = st.kurtosis( values )
    skew = st.skew( values )

    fig, axs = plt.subplots( ncols=2, figsize=(18*cm,8*cm))
    fig.suptitle( f'{var}: {n_zeros} zeros'  )
    axs[0].hist( values, bins=50 )
    axs[0].set_xlabel( 'values'  )
    axs[0].set_ylabel( 'count' )
    axs[0].set_title( f'skweness: {skew:.3f}, kurtosis: {kurtosis:.3f}')

    # values = values**expo
    values = np.log(values + 1)
    kurtosis = st.kurtosis( values )
    skew = st.skew( values )
    axs[1].hist( values, bins=50 )
    axs[1].set_xlabel( 'log(values+1)' )
    axs[1].set_ylabel( 'count' )
    axs[1].set_title( f'skweness: {skew:.3f}, kurtosis: {kurtosis:.3f}')

    fig.tight_layout()


## 2.4 autres vérifications

Certains `NumberofBuildings` sont particulièrement élevés, mais à priori OK.

In [ ]:
mask = 'NumberofBuildings'
df.loc[ df[mask] > 5, :].sort_values( by=mask )

Signification `NumberofFLoors` == 0 ? pas d'étage ?

Pour la `seattle chinese baptist church` 99 floors n'est pas cohérent (surtout en regardant l'image satellite): elle a donc été remplacée par 1 

In [ ]:
mask = 'NumberofFloors'
df.sort_values( by=mask, ascending=False ).iloc[:10,:]

PorpertyGFA : outliers ?

`university of washington - seattle campus` et `entire campus` correspondent à des valeurs `atypique` mais non aberrantes

In [ ]:
mask = 'PropertyGFATotal'
describe = df[mask].describe()
print(mask)
display(describe)

print('Q3 + 1.5*IQ = {:.3e}'.format( describe['75%'] 
            + 1.5*(describe['75%']-describe['25%']) ) )

df.sort_values( by=mask, ascending=False ).iloc[:5,:]

In [ ]:
mask = 'PropertyGFAParking'
describe = df[mask].describe()
print(mask)
display(describe.T)
df.sort_values( by=mask, ascending=False ).iloc[:5,:]

## 2.5 Categories "LargestPropertyUseType"

Liste des catégories

In [ ]:

tmp = pd.DataFrame( {'A':['a','a','b','c','c','c'],
                    'B':['b','b','c','d','d','e']})
display( tmp['A'].value_counts() )
display( tmp['B'].value_counts() )
display( tmp['A'].value_counts().add( tmp['B'].value_counts() , fill_value=0 ).astype(int)  )

In [ ]:
df['LargestPropertyUseType'].value_counts().add(
    df['SecondLargestPropertyUseType'].value_counts(), fill_value=0 ).add(
    df['ThirdLargestPropertyUseType'].value_counts(), fill_value=0 ).astype(int).sort_values()

In [ ]:
df.loc[ df['LargestPropertyUseType'].str.contains('other - entertainment/public assembly', na=False), 'PropertyName']

In [ ]:
masks = ['LargestPropertyUseType','SecondLargestPropertyUseType', 'ThirdLargestPropertyUseType']
for mask in masks:
    display( df.loc[df[mask].str.contains('recreation', na=False), mask].value_counts() )

In [ ]:
categories_keys = [ ('store', ['wholesale', 'mall', 'store', 'dealership'] ),
                    ('utility', ['fire', 'utility', 'police', 'courthouse', 'prison', 'bank']),
                    ('restaurant', ['food', 'restaurant']),
                    ('residential - hotel', ['residential', 'housing', 'hotel', 'dormitory']),
                    ('education', ['school', 'education', 'university']),
                    ('medical', ['care', 'hospital']),
                    ('office', ['financial office']),
                    ('entertainment/public assembly', ['theater','nightclub',
                                'recreation', 'swimming', 'performing arts',
                                'library','museum','meeting hall']),
                    ('lifestyle center', ['lifestyle', 'fitness']),
                    ('science', ['technology', 'laboratory']),
                    ('services', ['services'])
             ]
for (category,keys) in categories_keys :
    joint_keys = '|'.join(keys)
    for mask in masks:
        sr_loc = df[mask].str.contains( joint_keys , na=False )
        df.loc[ sr_loc, mask ] = category


value_counts = df['LargestPropertyUseType'].value_counts().add(
    df['SecondLargestPropertyUseType'].value_counts(), fill_value=0 ).add(
    df['ThirdLargestPropertyUseType'].value_counts(), fill_value=0 ).astype(int).sort_values()

print( f'{len(value_counts)} categories')
display( value_counts )

## 2.6 Year-built -> categories ?
à priori pas de relation directe évidente -> on peut séparer en plusieurs groupes

In [ ]:
X, Y = 'YearBuilt', 'SiteEUIWN(kBtu/sf)'

print('min:', df[X].min(), 'max:', df[X].max() )
print( np.arange( 1900, 2020, 20 ) )

bins_yearbuilt = [1900, 1920, 1934, 1943, 1960, 1980, 2000, 2020]
df['YearBuiltCateg'] = np.digitize( df[X], bins_yearbuilt )
display( df[[X, 'YearBuiltCateg']].sample(10) )

x = df[X].values
x.sort()
diff_x = x[1:]-x[:-1]
b_break = diff_x > 1
print( 'breaks begin at:', x[:-1][ b_break ])
print( 'breaks end at:', x[1:][ b_break ])

fig, ax = plt.subplots( figsize=(12*cm,6*cm) )
ax.plot( x[1:], diff_x, 'bo', markersize=3 )
ax.plot( x[1:][ b_break ], diff_x[ b_break ], 'ro', markersize=3 )
ax.set_xlabel(X)
ax.set_ylabel( 'year difference' )

fig, ax = plt.subplots( figsize=(12*cm,12*cm) )
ax.plot( df[X], df[Y], 'bo', markersize=3 )
ax_twinx = ax.twinx()
ax_twinx.plot( df[X], df['YearBuiltCateg'], 'yo', markersize=3 )

ax.set_xlabel( X )
ax.set_ylabel( Y )
fig.tight_layout()

***
# 3. PCA

In [ ]:
tmp = df.select_dtypes(include=[np.number])
features = [col for col in tmp.columns if not col in vars_to_delete]
print(features)

In [ ]:
X = df[features].dropna()
scaler_pca = preprocessing.StandardScaler()
X_scaled = scaler_pca.fit_transform(X) # fit and transform
idx = ["mean", "std"]
display( pd.DataFrame(X_scaled).describe().round(2).loc[idx, :] )

In [ ]:
from sklearn.decomposition import PCA
n_components = X_scaled.shape[1]
pca = PCA(n_components=n_components)

# entrainement
pca.fit(X_scaled)

In [ ]:
x_list = range(1, n_components+1)
scree = (pca.explained_variance_ratio_*100)
print('scree:', scree.round(2))
print('sum scree:', scree.sum().round(2))
fig, ax = plt.subplots( figsize=(12*cm,8*cm))
ax.bar( x_list, scree )
ax.set_xlabel("rang de l'axe d'inertie")
ax.set_ylabel("inertie (%)")
ax.set_title('Éboulis des valeurs propres')

ax = ax.twinx()
ax.set_ylabel("inertie (%)")

ax.plot( x_list, scree.cumsum(), c='r', marker='o', label='intertie cumulée')
ax.legend()

fig.tight_layout(pad=0.2)
# tools.savefig( fig, 'Figures/PCA/ebouli.pdf')

La variable `EnergySTARscore` n'impacte que peu les 6 premiers composantes de la PCA

In [ ]:
pcs = pd.DataFrame( pca.components_.transpose() )
pcs.index = features
columns = [f"F{i}" for i in x_list]
pcs.columns = columns

for i in range(6):
    key = f'F{i+1:}'
    display( pcs[[key]].sort_values( key, ascending=False ).T )

display( pcs.iloc[:,:6].round(2).T ) #.sort_values(by=indexes , ascending=False) )
fig, ax = plt.subplots(figsize=(26*cm, 12*cm))
sns.heatmap(pcs.iloc[:,:6].T*100, vmin=-100, vmax=100, annot=True, cmap="coolwarm", fmt="0.0f", annot_kws={"size": 8})
# fig.tight_layout( pad=0.2 )

In [ ]:
def correlation_graph(pca,
                      ij_F,
                      features,
                      ax=None) :
    """Affiche le graphe des correlations

    Positional arguments :
    -----------------------------------
    pca : sklearn.decomposition.PCA : notre objet PCA qui a été fit
    ij_F : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    features : list ou tuple : la liste des features (ie des dimensions) à représenter
    ax : axis sur lequel le graphique est tracé (default None -> est créé)
    """

    # Extrait x et y
    x,y=ij_F

    # Taille de l'image (en inches)
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 9))
    else:
        fig = ax.get_figure()

    # Pour chaque composante :
    for i in range(0, pca.components_.shape[1]):

        # Les flèches
        ax.arrow(0,0,
                pca.components_[x, i],
                pca.components_[y, i],
                head_width=0.07,
                head_length=0.07,
                width=0.02, )

        # Les labels
        ax.text(pca.components_[x, i] + 0.05*np.sign(pca.components_[x, i]),
                pca.components_[y, i] + 0.05*np.sign(pca.components_[y, i]),
                features[i])

    # Affichage des lignes horizontales et verticales
    ax.plot([-1, 1], [0, 0], color='grey', ls='--', zorder=0)
    ax.plot([0, 0], [-1, 1], color='grey', ls='--', zorder=0)

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel('F{} ({}%)'.format(x+1, round(100*pca.explained_variance_ratio_[x],1)))
    ax.set_ylabel('F{} ({}%)'.format(y+1, round(100*pca.explained_variance_ratio_[y],1)))

    # J'ai copié collé le code sans le lire
    ax.set_title("Cercle des corrélations (F{} et F{})".format(x+1, y+1))

    # Le cercle
    an = np.linspace(0, 2 * np.pi, 100)
    ax.plot(np.cos(an), np.sin(an), zorder=0 )  # Add a unit circle for scale

    # Axes et display
    ax.axis('equal')
    plt.show(block=False)
    return fig, ax

In [ ]:
for i in range(3):
    fig, ax = plt.subplots( figsize=(14*cm, 12*cm))
    
    correlation_graph( pca, [i+i,i+i+1], features, ax=ax )
    fig.tight_layout( )

***
# 4. Target

Présence de valeurs incohérentes (0 != sum autres variables) pour `SiteEUIWN(kBtu/sf)`

On regarde les dates des relevés

In [ ]:
display( df['DataYear'].value_counts() )

Toutes les mesures ont été faites la même année

In [ ]:
X, Y = 'SiteEUI(kBtu/sf)', 'SiteEUIWN(kBtu/sf)'
fig, ax = plt.subplots( figsize=(8*cm,8*cm))
ax.plot( df[X], df[Y], 'bo', markersize=3 )
ax.set_xlabel(X)
ax.set_ylabel(Y)
fig.tight_layout()

- `SiteEUI(kBtu/sf)` est basée sur les factures
- `SourceEUI(kbtu/sf)` : "the annual energy used to operate the property, including losses from generation, transmission, & distribution."

In [ ]:
X, Y = 'SiteEUI(kBtu/sf)', 'SourceEUI(kBtu/sf)'

lr = linear_model.LinearRegression()
lr.fit( df[X].values.reshape(-1,1), df[Y] )
print( f'estmated coefficient: {lr.coef_[0]:.3f}' )

fig, ax = plt.subplots( figsize=(8*cm,8*cm))
ax.plot( df[X], df[Y], 'bo', markersize=3 )

x = np.array([0, df[X].max()]).reshape(2,1)
ax.plot( x, lr.predict(x), 'r' )
ax.set_xlabel(X)
ax.set_ylabel(Y)
fig.tight_layout()

Vérification de conversion kWh -> Btu : OK

In [ ]:
X, Y = 'Electricity(kWh)', 'Electricity(kBtu)'
coef = 3.412142 # in [Btu] / [Wh]

lr = linear_model.LinearRegression()
lr.fit( df[X].values.reshape(-1,1), df[Y] )
print( f'estmated coefficient: {lr.coef_[0]:.3f}, theoretical: {coef:}' )

fig, ax = plt.subplots( figsize=(8*cm,8*cm))
ax.plot( df[X], df[Y], 'bo' )
ax.set_xlabel(X)
ax.set_ylabel(Y)
fig.tight_layout()

Vérification de conversion therms -> Btu : OK

In [ ]:
X, Y = 'NaturalGas(therms)', 'NaturalGas(kBtu)'
coef = 1e2 # in [kBtu]/[therms]

lr = linear_model.LinearRegression()
lr.fit( df[X].values.reshape(-1,1), df[Y] )
print( f'estmated coefficient: {lr.coef_[0]:.3f}, theoretical: {coef:}' )

fig, ax = plt.subplots( figsize=(8*cm,8*cm))
ax.plot( df[X], df[Y], 'bo' )
ax.set_xlabel(X)
ax.set_ylabel(Y)
fig.tight_layout()

Vérification de la somme des ressources utilisées, <span style="color:red"> variations dues à ?? </span>

`SiteEnergyUse(kBtu)` :The annual amount of energy consumed by the property from all sources of energy.

In [ ]:
X, Y = ['SteamUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)'], 'SiteEnergyUse(kBtu)'

X_values = df[X].sum(1)
diff = X_values - df[Y]

display( pd.DataFrame( {Y:df[Y], 'sum ressources':X_values} ).describe().T )

fig, ax = plt.subplots( figsize=(12*cm,8*cm))
ax.plot( X_values, df[Y], 'bo', markersize=4 )
ax.set_xlabel(r'$\sum$ steam,elec.,gas')
ax.set_ylabel(Y)

ax_twinx = ax.twinx()
ax_twinx.plot( X_values, np.abs(diff) / X_values * 100, 'ro', markersize=4 )
ax_twinx.set_ylabel('difference (%)', color='r')

fig.tight_layout()

On regarde à quoi correspondent les surfaces

Il y a clairement des incohérences entre les `LargestPropertyUseTypeGFA` et (`PropertyGFATotal`, `PropertyGFABuilding(s)`, `PropertyGFAParking`, `NumberofFloors`)

In [ ]:
x = df[['LargestPropertyUseTypeGFA',
        'SecondLargestPropertyUseTypeGFA',
        'ThirdLargestPropertyUseTypeGFA']].fillna(0.).sum(1)
# X = df['LargestPropertyUseTypeGFA']


Y1 = 'PropertyGFABuilding(s)'
Y2 = 'PropertyGFAParking'

Y = 'PropertyGFATotal'
y = df[Y]

# sr_loc = df['LargestPropertyUseType'] != 'parking'
# x = X.loc[sr_loc]
# y = y.loc[sr_loc]

# Y = 'GFA Building(s) + parking'
# y = df[Y1] + df[Y2]

# y = df['PropertyGFABuilding(s)'] * ( df['NumberofFloors'] +1 ) + df['PropertyGFAParking']
# Y = 'Surface à voir'

# diff = X - df[Y]
# display( pd.DataFrame( {Y:df[Y], 'sum surfaces':X_values} ).describe().T )

fig, ax = plt.subplots( figsize=(12*cm,8*cm))
ax.plot( x, y, 'bo', markersize=2 )
ax.plot( [0, y.max()], [0, y.max()], 'r', zorder=0 )
# ax.plot( [0,y.max()], [0,y.max()], 'r', label='ideal match' )
# ax.legend()

ax.set_xlabel(r'$\sum$ PrincipalUseTypeGFA')
ax.set_ylabel(Y)

# ax_twinx = ax.twinx()
# ax_twinx.plot( X_values, np.abs(diff) / X_values * 100, 'ro', markersize=4 )
# ax_twinx.set_ylabel('difference (%)', color='r')

fig.tight_layout()

In [ ]:
print( df.keys().tolist())
X, Y = 'SiteEnergyUse(kBtu)', 'SiteEUI(kBtu/sf)'

fig, ax = plt.subplots( figsize=(16*cm,8*cm))
X_values = df[X] / y
diff = X_values - df[Y]

lr = linear_model.LinearRegression()
lr.fit( X_values.values.reshape(-1,1), df[Y] )
# print( f'\nEstmated coefficient: {lr.coef_[0]:.3f}, ideal: {1:}' )

display( pd.DataFrame( {Y:df[Y], 're-calculated':X_values} ).describe().T )

ymax = df[Y].max()
xmax = X_values.max()
ax.set_title( f'coef.: {lr.coef_[0]:.3f}' )
ax.plot( [0,xmax], [0,xmax], 'r', label='ideal ratio' )
ax.plot( [0,xmax], lr.predict( np.array([0,xmax]).reshape(-1,1) ), 'y', label='estimated ratio' )
ax.plot( X_values, df[Y], 'bo', markersize=4 )
ax.set_xlabel( X + '\n/\n(estimated surface)')
ax.set_ylabel(Y)
ax.legend()

# ax_twinx = ax.twinx()
# ax_twinx.plot( X_values, np.abs(diff) / X_values * 100, 'ro', markersize=4 )
# ax_twinx.set_ylabel('difference (%)', color='r')

fig.tight_layout()

On prend la valeur "Weather Normalize".

In [ ]:
target = 'SiteEnergyUseWN(kBtu)'
sources = ['NaturalGas(kBtu)', 'Electricity(kBtu)',
            'SteamUse(kBtu)']

sr_0 = (df[target] == 0) | (df[target].isna())
print('number of elements target == 0 | isna:', (sr_0).sum())
# display( df.loc[sr_0,:])

y = df[target]
x = df[ sources ].fillna(0.).sum(1)

fig, ax = plt.subplots( figsize=(12*cm,8*cm))
ax.plot( x.loc[sr_0], y.loc[sr_0], 'yo', label='0 / NaN target')
ax.plot( [0, 4e8], [0,4e8], 'r', zorder=1 )
ax.annotate('x = y', [2.5e8, 2.3e8], va='center', ha='left', color='r')

ax.plot( x, y , 'bo', markersize=3 )
ax.legend()

Calcul des valeurs manquantes

In [ ]:
df[target] = x

sr_0 = (df[target] == 0) | (df[target].isna())
print('number of elements target == 0 | isna:', (sr_0).sum())
display( df.loc[sr_0,:])


In [ ]:
df.loc[sr_0,target] = df.loc[sr_0, 'SiteEnergyUse(kBtu)']

sr_0 = (df[target] == 0) | (df[target].isna())
print('number of elements target == 0 | isna:', (sr_0).sum())

Il y a un batiment (une "green structure") qui produit sa propre énergie et, à priori, plus que nécessaire 

In [ ]:
sr_neg = df[target] < 0

display( df.loc[sr_neg,:] )

In [ ]:

y = df[target].values

print( ( y + (1-y.min()) ).min() )
y_log = np.log( y + (1-y.min() ) )

fig, axs = plt.subplots( ncols=2, figsize=(16*cm,8*cm) )
n, bins, _ = axs[0].hist( y, bins=50 )
n, bins, _ = axs[1].hist( y_log, bins=50 )
axs[0].set_xlabel(target )
axs[1].set_xlabel( f'log( {target} )' )

axs[0].set_title( r'$\gamma_1$: {:.3f}, $\gamma_2$: {:.3f}'.format( st.skew(y), st.kurtosis(y) ) )
axs[1].set_title( r'$\gamma_1$: {:.3f}, $\gamma_2$: {:.3f}'.format( st.skew(y_log), st.kurtosis(y_log) ) )

fig.tight_layout()

log_target_bins_center = (bins[:-1] + bins[1:]) * 0.5 

## Scatter plots

In [ ]:

Xs = ['PropertyGFABuilding(s)', 'LargestPropertyUseTypeGFA']
hues = ['PrimaryPropertyType', 'LargestPropertyUseType']

# df[target]

Y = 'SiteEUIWN(kBtu/sf)'

sr_loc = (df[Y] != 0)
for X in Xs:
    sr_loc = sr_loc & (df[X]!=0)
tmp = df.loc[sr_loc, Xs + hues + [Y, 'Outlier']]
# tmp = df[[X,target, 'Outlier', hue]]

if True:
    # tmp[ Xs[1] ] /= tmp[ Xs[0] ]
    # sr_loc = tmp[Xs[1]] > 0.7
    sr_loc = tmp[hues[0]].str.contains('office')
    tmp = tmp.loc[sr_loc,:]


for X,hue in zip( Xs, hues ):
    g = sns.pairplot( data=tmp, vars=[X,Y], hue=hue,
        plot_kws={'s':6} )
    
    handles = g._legend_data.values()
    labels = g._legend_data.keys()
    g.fig.legend(handles=handles, labels=labels, loc=[0.2,0.65], ncol=1)

    g.fig.suptitle('linear values')
    g.fig.tight_layout()
    

    tmp[X] = np.log( tmp[X] )
    tmp[Y] = np.log( tmp[Y] )
    g = sns.pairplot( data=tmp, vars=[X,Y], hue=hue,
        plot_kws={'s':6} )
    # g.legend(bbox_to_anchor= (1.03, 1) )
    handles = g._legend_data.values()
    labels = g._legend_data.keys()
    g.fig.legend(handles=handles, labels=labels, loc=[0.35,0.65], ncol=1)
    g.legend.remove()
    g.fig.suptitle('log values')
    g.fig.tight_layout()
    
    break


In [ ]:
def ANOVA( df, hue, Y, transform=None ):
    groups = df[[Y,hue]].dropna().groupby(hue)[Y]
    index = pd.MultiIndex.from_tuples( [('shape','skew'), ('shape','kurtosis'),('shapiro','statistic'), ('shapiro','p-value')] )
    df_shape = pd.DataFrame( index = index)
    names = [name for name,yi in groups]
    for name, yi in groups:
        if not transform is None:
            yi = transform( yi )
        # print('\nname:', name)
        # print( st.skew(yi) )
        # print( st.kurtosis(yi) )
        # print( st.shapiro(yi) )
        # sr = pd.Series( [  ] ).astype(float)
        shapiro = st.shapiro(yi)
        df_shape[name] = [st.skew(yi), st.kurtosis(yi), shapiro.statistic, shapiro.pvalue]
    
    display(df_shape.round(3))

    levene = pg.homoscedasticity( df.dropna(), dv=Y, group=hue )
    display( levene )

    # tpl = tuple( [df.query( f'{hue} == "{name}"')[Y].dropna() for name in names ] )
    # print( st.levene( *tpl ) )
    

    display( pg.pairwise_tukey( dv=Y, between=hue, data=df.dropna() ) )

    # for i, (name,yi) in enumerate(groups):
    #     print('\n\n| {:} |\n\n'.format( '-'*20 ))
    #     if i+1 == len(names):
    #         break
    #     print(f'name: {name}')
    #     for j, (name_2,yi2) in enumerate(groups):
    #         if j <= i:
    #             continue
    #         print( f'\n{name} - {name_2}:' )
    #         stat, p = st.levene(yi, yi2)
    #         print( f'levene stat: {stat:.3f}, p-value: {p:.3f}' )
    #         print('std: {:.3f} - {:.3f}'.format( yi.std(), yi2.std() ) )
print('linear')
ANOVA( tmp, 'PrimaryPropertyType', 'SiteEUIWN(kBtu/sf)' )
# print('log')
# ANOVA( tmp, 'PrimaryPropertyType', 'SiteEUIWN(kBtu/sf)', lambda y:np.log( y + (1-y.min())) )

Les "office" semblent être des "small" offices

In [ ]:
mask = "PrimaryPropertyType"
df.loc[ df[mask] == 'office', mask] = 'small- and mid-sized office'

Levene test:
- `Null Hypothesis`: the `variances` are `equal` across all samples/groups
- `Alternative Hypothesis`:  the `variances` are `not equal` across all samples/groups

TODO: 
- Mesure kurosis 
- skewness
- scipy.stats.shapiro(x)

The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution.

Parameters:
xarray_like
Array of sample data.

Returns:
statisticfloat
The test statistic.

p-valuefloat
The p-value for the hypothesis test.

In [ ]:
import numpy as np
from scipy import stats
rng = np.random.default_rng()
x = stats.norm.rvs(loc=5, scale=3, size=100, random_state=rng)
shapiro_test = stats.shapiro(x)
shapiro_test
ShapiroResult(statistic=0.9813305735588074, pvalue=0.16855233907699585)
shapiro_test.statistic
0.9813305735588074
shapiro_test.pvalue
0.16855233907699585

In [ ]:

Xs = ['PropertyGFABuilding(s)', 'LargestPropertyUseTypeGFA']
hues = ['PrimaryPropertyType', 'LargestPropertyUseType']

# df[target]


sr_loc = (df[target] != 0)
for X in Xs:
    sr_loc = sr_loc & (df[X]!=0)
tmp = df.loc[sr_loc, Xs + hues + [target, 'Outlier']]
# tmp = df[[X,target, 'Outlier', hue]]

if True:
    # tmp[ Xs[1] ] /= tmp[ Xs[0] ]
    # sr_loc = tmp[Xs[1]] > 0.7
    sr_loc = tmp[hues[0]].str.contains('office')
    tmp = tmp.loc[sr_loc,:]

describe = tmp.groupby(mask).describe()
display( describe.loc[ :, idx[:, ['mean', 'std']  ] ].round(2) )

# Levene's test
mask = 'PrimaryPropertyType'
# tpl = tuple( [ tmp.query('{:} == "{:}"'.format( mask, grp ) ) for grp in tmp[mask].unique() ] )
# tpl_grps = ( tmp.query( '{:} == "{:}"'.format( mask, grp ) for grp in tmp[mask].unique() ) )

levene = pg.homoscedasticity( tmp, dv=target, group=mask )
display( levene )
print(f'equal var hypothesis validated : {levene.iloc[0,2]}')

for X,hue in zip( Xs, hues ):
    break
    sns.pairplot( data=tmp, vars=[X,target], hue=hue,
        plot_kws={'s':3} )

    tmp[X] = np.log( tmp[X] )
    tmp[target] = np.log( tmp[target] )
    sns.pairplot( data=tmp, vars=[X,target], hue=hue,
        plot_kws={'s':3} )
    
    break


# Analyse des features

## utilisation "office"

On regarde les différents types d'utilisation `office`

In [ ]:
vars =['PrimaryPropertyType',
        'LargestPropertyUseType']
        # ,
        # 'SecondLargestPropertyUseType',
        # 'ThirdLargestPropertyUseType']

sr_loc = df[vars[0]].str.contains('office', na=False)

for var in vars:
    sr_loc = sr_loc | df[var].str.contains('office', na=False)

for var in vars:
    print(f'{var}:')
    display( df.loc[sr_loc,var].value_counts() )

In [ ]:
fig, axs = plt.subplots( figsize=(16*cm,20*cm), nrows=6, sharex=True)
axs_2 = [axs[0] , axs[2], axs[3], axs[4], axs[5], axs[1] ]
df.loc[sr_loc, :].plot.hist( ax=axs_2, column=[target], by='PrimaryPropertyType' , bins=50 )
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots( nrows=3, sharex=True)
df.loc[sr_loc, :].plot.hist( ax=ax, column=[target], by='LargestPropertyUseType' , bins=50 )
fig.tight_layout()

À priori, il est possible de regrouper tous les type d'`offices` ensemble: il y a peut de "financial" et "medical" offices, et ne semblet pas créer de mode singulièrement différent du mode principale.

In [ ]:
vars =['PrimaryPropertyType',
        'LargestPropertyUseType',
        'SecondLargestPropertyUseType',
        'ThirdLargestPropertyUseType']

for var in vars:
    sr_loc = df[var].str.contains( 'office', na=False )
    df.loc[sr_loc, var] = 'office'

# Date and location

## City

In [ ]:
for key in ['City', 'State', 'ZipCode',
            'DataYear', 'YearBuilt', 
            'CouncilDistrictCode', 'Neighborhood',
            'Latitude', 'Longitude']:
    display( df[key].value_counts() )

La variable city n'est pas utile car elle ne présente qu'une unique valeur: seattle.

In [ ]:
col_to_drop = ['City', 'State', 'DataYear']
df = df.drop( columns=col_to_drop )

# Location

In [ ]:
x, y = df['Longitude'], df['Latitude']
x -= x.mean()
y -= y.mean()
values = np.log( df[target] )

sr_loc = df['LargestPropertyUseType'] == 'office'
x, y, values = x[sr_loc], y[sr_loc], values[sr_loc]

fig, ax = plt.subplots()
scttr = ax.scatter( x, y , s=3, c=values )
plt.colorbar( scttr , label=target + ' (log)' )

ANOVA Zipcode - energy

In [ ]:
from scipy import stats

def ANOVA(x,y):
    moyenne_y = y.mean()
    classes = {}
    for classe in x.unique():
        yi_classe = y[x==classe]
        ymean_class = yi_classe.mean()
        n_i = len(yi_classe)
        classes[classe] = {'n_i': n_i,
                        'moyenne_classe': ymean_class,
                        's_i': (yi_classe**2).sum() - n_i*ymean_class**2 }
    k = len(classes) # number of condittions
    N = len(y) # number of values

    # Degree of Freedom (DF)
    DFbetween = k - 1
    DFwithin = N - k
    DFtotal = N - 1

    # Sum of Squares (SS)
    SStotal = (y**2).sum() - N*moyenne_y**2
    SSbetween = sum([c['n_i']*(c['moyenne_classe']-moyenne_y)**2 for c in classes.values()])
    SSwithin = sum( [c['s_i'] for c in classes.values()] )

    # Meas Square
    MSbetween = SSbetween/DFbetween
    MSwithin = SSwithin/DFwithin

    F = MSbetween/MSwithin
    p = stats.f.sf( F, DFbetween, DFwithin )
    eta_sqrd = SSbetween/SStotal

    omega_sqrd = (SSbetween - (DFbetween * MSwithin))/(SStotal + MSwithin)

    return F,p, eta_sqrd, omega_sqrd, classes

In [ ]:
X,Y = 'ZipCode', target
df_tmp = df[[X,Y]].dropna()

df_describe = df_tmp[[Y]].describe()

# Q3 + 1.5*IQ
vlim = df_describe[Y]['75%']*2.5 - 1.5*df_describe[Y]['25%']
print('Q3 + 1.5*IQ: {:.2f}'.format( vlim ))

# ommition des valeurs abérrantes
df_tmp = df_tmp.loc[df[Y] < vlim, :]


 

df_describe = pd.concat((df_describe.T, df_tmp[[Y]].describe().T))
df_describe.index = ['before crop', 'after crop']

display( df_describe.round(3) )

F,p, eta_sqrd, omega_sqrd, classes = ANOVA( df_tmp[X], df_tmp[Y] )

print( f'F: {F:.3f},', 'p', p, 'eta_sqrd', eta_sqrd, 'omega_sqrd', omega_sqrd)


import statsmodels.api as sm
from statsmodels.formula.api import ols

print('columns:', df_tmp.columns)
# help( ols )
# mod = ols( Y + ' ~ ' + X, data=df_tmp).fit()
                
# aov_table = sm.stats.anova_lm(mod, typ=2)
# aov_table[['eta_sqrd', 'omega_sqrd']] = np.nan
# aov_table.loc[ X, 'eta_sqrd' ] = eta_sqrd
# aov_table.loc[ X, 'omega_sqrd' ] = omega_sqrd
# display( aov_table )
# print( tools.df2ltx(aov_table.round(3)) )

print('eta_sqrd: {:.3f}'.format( eta_sqrd ) )



props = {'color':'w', 'linewidth':2}


fig, ax = plt.subplots( figsize=(12*cm, 8*cm) )
df_tmp.boxplot( column=Y, by=X, ax=ax, sym='bo', 
            boxprops=props, whiskerprops=props,
            capprops=props, medianprops=props,
            widths=0.5)
ax.set_ylabel(Y.replace('_100g', ' (%)'))
ax.set_xlabel(X.replace('_', ' '))
# ax.set_title(r'$\eta^2={{{:.3f}}}$'.format(eta_sqrd))
ax.set_title('')
fig.suptitle(None)
fig.tight_layout( pad=1 )
# tools.savefig( fig, 'Figures/ANOVA/boxes.pdf')

## Tax Parcel IdentificationNumber

In [ ]:
df['TaxParcelIdentificationNumber'].value_counts()

Vérification doublons

In [ ]:
df['OSEBuildingID'].duplicated().sum()

***
# Définition d'un dataset "numérique"

In [ ]:
df_num = pd.DataFrame()
features = ['Longitude', 'Latitude', 'YearBuilt',
        'NumberofBuildings', 'NumberofFloors',
        'PropertyGFATotal', 'PropertyGFAParking']
df_num[features] = df[features].values

df_num.isna().sum()

In [ ]:
arr_0 = df['LargestPropertyUseType'].unique()
arr_1 = df['SecondLargestPropertyUseType'].unique()
arr_2 = df['ThirdLargestPropertyUseType'].unique()

arr = np.concatenate( (arr_0, arr_1, arr_2 ))
print(arr.size)
PropertyUseTypes = [val for i, val in enumerate(arr) if (not val in arr[:i]) & ~isinstance(val, float)]
print( len(PropertyUseTypes) )
X_PropertyUseTypeGFA = np.zeros( (len(df), len(PropertyUseTypes)) )
for i, index in enumerate(df.index):
    for suffix in ['','Second','Third']:
        usetype = df.at[index, suffix + 'LargestPropertyUseType']
        if isinstance( usetype, float): # test if isnan
            break
        j = PropertyUseTypes.index( usetype )
        X_PropertyUseTypeGFA[i,j] += df.at[index, 'LargestPropertyUseTypeGFA']

features += ['GFA ' + usetype for usetype in PropertyUseTypes]

In [ ]:
X = np.hstack( (df_num.values,X_PropertyUseTypeGFA) )
y = df[target]
sr_loc = ~(y.isna()) & (y != 0)

X = X[sr_loc.values]
y = np.log( y.values[sr_loc.values] )

def bin_to_class( val, bins ):
    for i, bin in enumerate(bins):
        if val < bin:
            return i
    return len(bins) 
y_classes = np.array([ bin_to_class( val, log_target_bins_center ) for val in y ])

indexes = sr_loc[sr_loc].index


## PCA

In [ ]:
scaler_pca = preprocessing.StandardScaler()
X_scaled = scaler_pca.fit_transform(X) # fit and transform
idx = ["mean", "std"]
display( pd.DataFrame(X_scaled).describe().round(2).loc[idx, :] )

In [ ]:
from sklearn.decomposition import PCA
n_components = X_scaled.shape[1]
pca = PCA(n_components=n_components)

# entrainement
pca.fit(X_scaled)

In [ ]:
x_list = range(1, n_components+1)
scree = (pca.explained_variance_ratio_*100)
print('scree:', scree.round(2))
print('sum scree:', scree.sum().round(2))
fig, ax = plt.subplots( figsize=(12*cm,8*cm))
ax.bar( x_list, scree )
ax.set_xlabel("rang de l'axe d'inertie")
ax.set_ylabel("inertie (%)")
ax.set_title('Éboulis des valeurs propres')

ax = ax.twinx()
ax.set_ylabel("inertie (%)")

ax.plot( x_list, scree.cumsum(), c='r', marker='o', label='intertie cumulée')
ax.legend()

fig.tight_layout(pad=0.2)
# tools.savefig( fig, 'Figures/PCA/ebouli.pdf')


In [ ]:
pcs = pd.DataFrame( pca.components_.transpose() )
pcs.index = features
columns = [f"F{i}" for i in x_list]
pcs.columns = columns

for i in range(20):
    key = f'F{i+1:}'
    display( pcs[[key]].sort_values( key, ascending=False ).T )

display( pcs.round(2).T ) #.sort_values(by=indexes , ascending=False) )
fig, ax = plt.subplots(figsize=(18*cm, 8*cm))
sns.heatmap(pcs.T, vmin=-1, vmax=1, annot=True, cmap="coolwarm", fmt="0.2f")
fig.tight_layout( pad=0.2 )

In [ ]:
def correlation_graph(pca,
                      ij_F,
                      features,
                      ax=None) :
    """Affiche le graphe des correlations

    Positional arguments :
    -----------------------------------
    pca : sklearn.decomposition.PCA : notre objet PCA qui a été fit
    ij_F : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    features : list ou tuple : la liste des features (ie des dimensions) à représenter
    ax : axis sur lequel le graphique est tracé (default None -> est créé)
    """

    # Extrait x et y
    x,y=ij_F

    # Taille de l'image (en inches)
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 9))
    else:
        fig = ax.get_figure()

    # Pour chaque composante :
    for i in range(0, pca.components_.shape[1]):

        # Les flèches
        ax.arrow(0,0,
                pca.components_[x, i],
                pca.components_[y, i],
                head_width=0.07,
                head_length=0.07,
                width=0.02, )

        # Les labels
        ax.text(pca.components_[x, i] + 0.05*np.sign(pca.components_[x, i]),
                pca.components_[y, i] + 0.05*np.sign(pca.components_[y, i]),
                features[i])

    # Affichage des lignes horizontales et verticales
    ax.plot([-1, 1], [0, 0], color='grey', ls='--', zorder=0)
    ax.plot([0, 0], [-1, 1], color='grey', ls='--', zorder=0)

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel('F{} ({}%)'.format(x+1, round(100*pca.explained_variance_ratio_[x],1)))
    ax.set_ylabel('F{} ({}%)'.format(y+1, round(100*pca.explained_variance_ratio_[y],1)))

    # J'ai copié collé le code sans le lire
    ax.set_title("Cercle des corrélations (F{} et F{})".format(x+1, y+1))

    # Le cercle
    an = np.linspace(0, 2 * np.pi, 100)
    ax.plot(np.cos(an), np.sin(an), zorder=0 )  # Add a unit circle for scale

    # Axes et display
    ax.axis('equal')
    plt.show(block=False)
    return fig, ax

In [ ]:
for i in range(3):
    fig, ax = plt.subplots( figsize=(14*cm, 12*cm))
    
    correlation_graph( pca, [i+i,i+i+1], features, ax=ax )
    fig.tight_layout( )

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

_, _, i_train, i_test = model_selection.train_test_split( range(len(y)) , 
                                    range(len(y)), train_size=0.8)

X_train = X[i_train, :]
X_test = X[i_test, :]
y_train = y_classes[i_train]
y_test = y_classes[i_test]

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# help( RandomForestClassifier )
rfc = RandomForestClassifier( n_estimators=100, n_jobs=-1, oob_score=True )
rfc.fit( X_train, y_train )

pred = rfc.predict(X_test)
print("accuracy: {:.2f}%".format(100* metrics.accuracy_score(y_test, pred)) )
print('score: {:.2f}'.format( 100*rfc.score( X_test, y_test) ))

argsort = y_test.argsort()
fig, ax = plt.subplots()
ax.plot( pred[argsort], 'o' )
ax.plot( y_test[argsort], 'r' )

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# X_train = X[i_train, :]
# X_test = X[i_test, :]
y_train = y[i_train]
y_test = y[i_test]


rfr = RandomForestRegressor( n_estimators=100, n_jobs=-1, oob_score=True )
rfr.fit( X_train, y_train )
print("score: {:.2f}".format(100* rfr.score(X_test, y_test)) )
y_pred = rfr.predict( X_test )
# help( rfr.score )

argsort = y_test.argsort()
fig, axs = plt.subplots(nrows=2)
axs[0].plot( y_pred[argsort], 'o' )
axs[0].plot( y_test[argsort], 'r' )

y_train = np.exp( y_train )
y_test = np.exp( y_test )

rfr = RandomForestRegressor( n_estimators=100, n_jobs=-1, oob_score=True )
rfr.fit( X_train, y_train )
print("score: {:.2f}".format(100* rfr.score(X_test, y_test)) )
y_pred = rfr.predict( X_test )

axs[1].semilogy( y_pred[argsort], 'o' )
axs[1].plot( y_test[argsort], 'r' )

***
# Random Forest

In [ ]:
vars_X = ['BuildingType', 'PrimaryPropertyType',
        'ZipCode', 'TaxParcelIdentificationNumber',
        'CouncilDistrictCode', 'Neighborhood',
        'Latitude', 'Longitude',
        'YearBuilt', 'NumberofBuildings',
        'NumberofFloors', 'PropertyGFATotal',
        'PropertyGFAParking', 'PropertyGFABuilding(s)',
        'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
        'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA',
        'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA'
        ]


X = df[vars_X].copy()
keys = [ 'LargestPropertyUseType', 'SecondLargestPropertyUseType',
        'ThirdLargestPropertyUseType' ]

X[ keys ] = X[keys].fillna( 'None' )

keys = [ 'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA',
        'ThirdLargestPropertyUseTypeGFA' ]
X[ keys ] = X[keys].fillna( 0. )

display( X.isna().sum())
y = df[target].copy()

def bin_to_class( val, bins ):
    for i, bin in enumerate(bins):
        if val < bin:
            return i
    return len(bins) 
y_classes = pd.Series( [ bin_to_class( val, log_target_bins_center ) for val in y ] , index=y.index )



In [ ]:
from sklearn.ensemble import RandomForestClassifier

_, _, index_train, index_test = model_selection.train_test_split( range(len(df)) , 
                                    X.index, train_size=0.8)

X_train = X.loc[index_train, :]
X_test = X.loc[index_test, :]
y_train = y_classes.loc[index_train]
y_test = y_classes.loc[index_test]

help( RandomForestClassifier )
rfc = RandomForestClassifier( n_estimators=100, n_jobs=-1, oob_score=True )
rfc.fit( X_train, y_train )

In [ ]:
from sklearn.ensemble import RandomForestRegressor

_, _, index_train, index_test = model_selection.train_test_split( range(len(df)) , 
                                    X.index, train_size=0.8)

X_train = X.loc[index_train, :]
X_test = X.loc[index_test, :]
y_train = y.loc[index_train]
y_test = y.loc[index_test]

rfr = RandomForestRegressor( n_estimators=100, n_jobs=-1, oob_score=True )
rfr.fit( X_train, y_train )

# Quels types d'utilisation ?
- il est probablement préférable de ne garder que le `PrimaryPropertyType`
- il y a une `incohérence` dans le tri `large` | `small- or mid-sized` office : les histrogrammes se recoupent
- <span style="color:red"> residence hall/domitory ? </span>
- <span style="color:red"> LargestPropertyUseType : other - lodging/residential ? </span>
- <span style="color:red"> Faut-il faire quelques groupes ? Offiche + others ? </span>

In [ ]:
vars = ['PrimaryPropertyType',
        'LargestPropertyUseType',
        'SecondLargestPropertyUseType',
        'ThirdLargestPropertyUseType']

for var in vars:
    value_counts = df[var].value_counts()
    display( value_counts )
    fig, ax = plt.subplots()
    value_counts.plot( kind='pie' , ax=ax, autopct='%.1f%%' )
    ax.set_title( var )
    ax.set_ylabel('')
    fig.tight_layout(pad=0.2)


fig, ax = plt.subplots()

print('small- and mid-sized office')
sr_loc = df[vars[0]].str.contains('mid-sized office')
display( df.loc[sr_loc, ['LargestPropertyUseTypeGFA'] ].describe() )

values_small_med = np.log( df.loc[sr_loc, 'LargestPropertyUseTypeGFA' ].values )


var_surf = 'LargestPropertyUseTypeGFA'

print('large office')
sr_loc = df[vars[0]].str.contains('large office')
display( df.loc[sr_loc, [var_surf] ].describe() )

values_large = np.log( df.loc[sr_loc, var_surf] )

ax.hist( [values_small_med, values_large],
        bins=50, label=['small- and mid-sized', 'large'] )

ax.plot([],[], 'r', label='size not referenced')

ax.legend()
ax.set_title('histogram of the surface used as office')

sr_loc = df[vars[0]] == 'office'
for value in df.loc[sr_loc, var_surf]:
    ax.plot( [np.log(value),]*2, [0,10] , 'r')

## Observation office size - targets

In [ ]:
targets = ['TotalGHGEmissions']